# Análisis de sentimientos
En este cuaderno utilizaremos un modelo especializado en detectar *sentimientos* de los comentarios de un juego se Steam.


In [55]:
import os
import requests
import json

### Obtención de lista de juegos y su appid

In [56]:
# Si no tenemos el archivo steam-games.json lo solicitamos a la api y lo generamos.
if not os.path.exists('steam-games.json'):
  # Url del endpoint
  gameListUrl = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"

  # Realizar la solicitud GET
  response = requests.get(gameListUrl)

  # Verificar si la solicitud fue exitosa
  if response.status_code == 200:
    # Obtener el contenido JSON de la respuesta
    data = response.json()
    
    # Guardar el JSON en un archivo
    with open('steam-games.json', 'w') as file:
      json.dump(data, file)
    
    # Cierro el archivo
    file.close()
  else:
    print("Error al realizar la solicitud:", response.status_code)

### Buscador para obtener el appid dado el nombrede un juego.

Si ya sabemos el appid del juego podemos saltarnos esta celda e introducirlo en el input de la siguente.

In [74]:
def buscar_appid(game_name):
  # Cargar los datos del archivo JSON
  with open('steam-games.json', 'r') as file:
    data = json.load(file)

  # Cerrar el archivo
  file.close()

  # Obtener la lista de juegos
  games = data['applist']['apps']

  # Buscar coincidencias exactas al inicio del nombre
  matches_start = [game for game in games if game['name'].lower().startswith(game_name.lower())]

  # Si no hay coincidencias exactas al inicio, buscar coincidencias en cualquier parte del nombre
  if len(matches_start) == 0:
    matches_contain = [game for game in games if game_name.lower() in game['name'].lower()]
  else:
    matches_contain = []

  matches = matches_start + matches_contain

  # Si no hay coincidencias, solicitar de nuevo
  while len(matches) == 0:
    print("No se encontraron coincidencias exactas. Inténtalo de nuevo.\n")
    game_name = input("Introduce el nombre del juego: ")
    matches_start = [game for game in games if game['name'].lower().startswith(game_name.lower())]
    if len(matches_start) == 0:
      matches_contain = [game for game in games if game_name.lower() in game['name'].lower()]
    else:
      matches_contain = []
    matches = matches_start + matches_contain

  if len(matches) == 1 and matches[0]['name'].lower() == game_name.lower():
    appid = matches[0]['appid']
    print(f"El appid del juego '{game_name}' es: {matches[0]['appid']}")
  else:
    print("Se encontraron varias coincidencias:")
    for i, match in enumerate(matches[:5]):
      if i == len(matches[:5]) - 1:
        print(f"{i + 1}. {match['name']}\n")
      else:
        print(f"{i + 1}. {match['name']}")

    # Solicitar al usuario que elija una opción
    choice = int(input("Elige el número correspondiente al juego: ")) - 1

    # Devolver el appid del juego elegido
    if 0 <= choice < len(matches):
      print(f"El appid del juego '{matches[choice]['name']}' es: {matches[choice]['appid']}")
      appid = matches[choice]['appid']
    else:
      print("Opción no válida.")
      appid = None

  return appid

### Extracción de reviews de dicho juego
Extraemos las reviews más relevantes  de los juegos, extraemos un máximo de 100 reviews

In [77]:
def obtener_reviews(appid):
    gameReviewsUrl = f"https://store.steampowered.com/appreviews/{appid}?json=1&language=english&num_per_page=100"
    response = requests.get(gameReviewsUrl)

    if response.status_code == 200:
        data = response.json()
    else:
        print("Error al realizar la solicitud:", response.status_code)
        return None, None

    return data, gameReviewsUrl

def solicitar_appid():
    choice = input("Introduce el appid del juego o el nombre del juego: ")

    if choice.isdigit():
        return int(choice)
    else:
        return buscar_appid(choice)

# Bucle para solicitar appid o nombre del juego hasta encontrar uno con reviews o el usuario introduzca -1
while True:
    # Solicitar appid o nombre del juego
    appid = solicitar_appid()

    # Verificar si el usuario desea salir
    if appid == -1:
        print("Saliendo del programa.")
        break

    # Obtener reviews
    data, gameReviewsUrl = obtener_reviews(appid)

    # Comprobar si el juego tiene reviews
    if data['query_summary']['total_reviews'] == 0:
        print("El juego no tiene reviews. Por favor, introduce otro appid o busca el appid por el nombre del juego.")
    else:
        reviews = data['reviews']
        num_reviews = len(reviews)
        if num_reviews == 100:
            print(f"Se han cargado {num_reviews} reviews (max).")
        else:
            print(f"Se han cargado {num_reviews} reviews.")
        break


Se encontraron varias coincidencias:
1. Fox and Shadow
2. Fox Goes Hunting ™
3. Foxnox
4. Fox of the moon
5. Foxblade Prologue

El appid del juego 'Fox and Shadow' es: 2134370
El juego no tiene reviews. Por favor, introduce otro appid o busca el appid por el nombre del juego.
Se encontraron varias coincidencias:
1. Fox and Shadow
2. Fox Goes Hunting ™
3. Foxnox
4. Fox of the moon
5. Foxblade Prologue

El appid del juego 'Fox Goes Hunting ™' es: 1720860
Se han cargado 2 reviews.


### Detección de sentimientos
En este apartado analizamos los sentimientos de cada una de las reviews cargadas y finalmente estableceremos un valor promedio para valorar el juego.

In [59]:
#%pip install torch
#%pip install transformers

In [60]:
from transformers import pipeline

cls = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [78]:
# Truncar las reviews a un máximo de 512 caracteres
max_length = 512
truncated_reviews = [review['review'][:max_length] for review in reviews]

# Obtener los resultados de análisis de sentimiento de cada review
results = [cls(review)[0] for review in truncated_reviews]

# Calculo el peso de cada sentimiento y el conteo de cada categoría
sentiment_scores = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0}
sentiment_counts = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0}
for result in results:
    label = result['label']
    score = result['score']
    sentiment_scores[label] += score
    sentiment_counts[label] += 1

# Calculo el sentimiento predominante
total_score = sum(sentiment_scores.values())
weighted_sentiment = {k: v / total_score for k, v in sentiment_scores.items()}

# Sacamos un sentimiento medio en forma de texto
if weighted_sentiment['POSITIVE'] >= 0.85:
    overall_sentiment = 'VERY POSITIVE 🎉'
elif weighted_sentiment['POSITIVE'] >= 0.5:
    overall_sentiment = 'POSITIVE 👍🏻'
elif weighted_sentiment['NEGATIVE'] >= 0.85:
    overall_sentiment = 'VERY NEGATIVE 💩'
elif weighted_sentiment['NEGATIVE'] >= 0.5:
    overall_sentiment = 'NEGATIVE 👎🏻'
else:
    overall_sentiment = 'NEUTRAL 🤷🏻‍♀️'

# Mostramos los resultados
print(f"Sentimiento predominante: {overall_sentiment}")
print(f"\nConteo de reviews por categoría:")
print(f"Positivas: {sentiment_counts['POSITIVE']}")
print(f"Negativas: {sentiment_counts['NEGATIVE']}")
print(f"Neutrales: {sentiment_counts['NEUTRAL']}")

Sentimiento predominante: VERY NEGATIVE 💩

Conteo de reviews por categoría:
Positivas: 0
Negativas: 2
Neutrales: 0
